# Tạo DataFrame mẫu
data = [
    ("Phòng A", "Nhân viên 1", 3000),
    ("Phòng A", "Nhân viên 2", 4600),
    ("Phòng A", "Nhân viên 3", 4100),
    ("Phòng B", "Nhân viên 4", 2500),
    ("Phòng B", "Nhân viên 5", 3600),
    ("Phòng B", "Nhân viên 6", 3300),
    ("Phòng C", "Nhân viên 7", 5200),
    ("Phòng C", "Nhân viên 8", 5400),
]
columns = ["Phòng ban", "Tên nhân viên", "Lương"]
1. Tính số thứ tự của mỗi nhân viên khi được sắp xếp theo lương trong phòng ban của họ.
   
2. Tính mức lương trung bình trong từng phòng ban và mức chênh lệch lương của mỗi nhân viên so với mức trung bình đó.

3. Xác định người có mức lương cao nhất và thấp nhất trong mỗi phòng ban.

4. Tính mức lương lớn thứ hai trong từng phòng ban.

5. Sử dụng hàm lead() để so sánh mức lương của mỗi nhân viên với nhân viên tiếp theo trong cùng phòng ban dựa trên thứ tự sắp xếp lương.

6. Sử dụng hàm lag() để so sánh mức lương của mỗi nhân viên với nhân viên trước đó trong cùng phòng ban.

7. Xác định phòng ban nào có tổng lương cao nhất.

8. Tính tổng lương lũy kế của từng nhân viên trong phòng ban của họ, sắp xếp theo thứ tự lương tăng dần.

9. Sử dụng cume_dist để xác định tỷ lệ phần trăm các nhân viên có mức lương thấp hơn so với mỗi nhân viên trong phòng ban của họ.

10. Sử dụng hàm percent_rank() để xác định xếp hạng phần trăm của mỗi nhân viên trong phòng ban của họ dựa trên mức lương. 

### 0. khoi tao dataframe

In [1]:
from pyspark.sql import SparkSession

In [2]:
from pyspark.sql.window import Window

In [3]:
from pyspark.sql.functions import *

In [4]:
ss = SparkSession.builder \
    .appName("window") \
    .master("local[*]") \
    .getOrCreate()

In [5]:
data = [
    ("Phòng A", "Nhân viên 1", 3000),
    ("Phòng A", "Nhân viên 2", 4600),
    ("Phòng A", "Nhân viên 3", 4100),
    ("Phòng B", "Nhân viên 4", 2500),
    ("Phòng B", "Nhân viên 5", 3600),
    ("Phòng B", "Nhân viên 6", 3300),
    ("Phòng C", "Nhân viên 7", 5200),
    ("Phòng C", "Nhân viên 8", 5400),
]
columns = ["phong_ban", "ten_nhan_vien", "luong"]
salary_df = ss.createDataFrame(data, columns)

In [6]:
salary_df.show()

+---------+-------------+-----+
|phong_ban|ten_nhan_vien|luong|
+---------+-------------+-----+
|  Phòng A|  Nhân viên 1| 3000|
|  Phòng A|  Nhân viên 2| 4600|
|  Phòng A|  Nhân viên 3| 4100|
|  Phòng B|  Nhân viên 4| 2500|
|  Phòng B|  Nhân viên 5| 3600|
|  Phòng B|  Nhân viên 6| 3300|
|  Phòng C|  Nhân viên 7| 5200|
|  Phòng C|  Nhân viên 8| 5400|
+---------+-------------+-----+



### 1. Tính số thứ tự của mỗi nhân viên khi được sắp xếp theo lương trong phòng ban của họ.

In [64]:
sx_luong_theo_phong_ban_giam_dan_window = Window.partitionBy("phong_ban").orderBy(desc("luong"))
sx_luong_theo_phong_ban_tang_dan_window = Window.partitionBy("phong_ban").orderBy(asc("luong"))

In [65]:
rank_demo = row_number().over(sx_luong_theo_phong_ban_giam_dan_window) # row_number() la 1 ham cua so danh so thu tu, su dung cho 1 cua so. tra ve cot

In [66]:
sx_theo_luong_df = salary_df.withColumn("rank_luong_theo_phong_giam_dan", rank_demo)
sx_theo_luong_tang_dan_df = salary_df.withColumn("rank_luong_theo_phong_tang_dan", row_number().over(sx_luong_theo_phong_ban_tang_dan_window))

In [62]:
sx_theo_luong_df.show()
sx_theo_luong_tang_dan_df.show()

+---------+-------------+-----+------------------------------+
|phong_ban|ten_nhan_vien|luong|rank_luong_theo_phong_giam_dan|
+---------+-------------+-----+------------------------------+
|  Phòng A|  Nhân viên 2| 4600|                             1|
|  Phòng A|  Nhân viên 3| 4100|                             2|
|  Phòng A|  Nhân viên 1| 3000|                             3|
|  Phòng B|  Nhân viên 5| 3600|                             1|
|  Phòng B|  Nhân viên 6| 3300|                             2|
|  Phòng B|  Nhân viên 4| 2500|                             3|
|  Phòng C|  Nhân viên 8| 5400|                             1|
|  Phòng C|  Nhân viên 7| 5200|                             2|
+---------+-------------+-----+------------------------------+

+---------+-------------+-----+------------------------------+
|phong_ban|ten_nhan_vien|luong|rank_luong_theo_phong_tang_dan|
+---------+-------------+-----+------------------------------+
|  Phòng A|  Nhân viên 1| 3000|                       

### 2. Tính mức lương trung bình trong từng phòng ban và mức chênh lệch lương của mỗi nhân viên so với mức trung bình đó.


In [25]:
window_phong_ban = Window.partitionBy("phong_ban")

In [26]:
avg_demo =avg(salary_df["luong"]).over(window_phong_ban)

In [29]:
luong_tb_theo_phong_ban_df = salary_df.withColumn("luong_tb_theo_phong_ban", avg_demo)

In [31]:
luong_tb_theo_phong_ban_df.show()

+---------+-------------+-----+-----------------------+
|phong_ban|ten_nhan_vien|luong|luong_tb_theo_phong_ban|
+---------+-------------+-----+-----------------------+
|  Phòng A|  Nhân viên 1| 3000|                 3900.0|
|  Phòng A|  Nhân viên 2| 4600|                 3900.0|
|  Phòng A|  Nhân viên 3| 4100|                 3900.0|
|  Phòng B|  Nhân viên 4| 2500|     3133.3333333333335|
|  Phòng B|  Nhân viên 5| 3600|     3133.3333333333335|
|  Phòng B|  Nhân viên 6| 3300|     3133.3333333333335|
|  Phòng C|  Nhân viên 7| 5200|                 5300.0|
|  Phòng C|  Nhân viên 8| 5400|                 5300.0|
+---------+-------------+-----+-----------------------+



In [39]:
chenh_luong_so_voi_luong_trung_binh_col = round(luong_tb_theo_phong_ban_df["luong"] - luong_tb_theo_phong_ban_df["luong_tb_theo_phong_ban"], 2)

In [40]:
chenh_luong_so_voi_luong_trung_binh_df = luong_tb_theo_phong_ban_df.withColumn("chenh_luong_so_voi_luong_trung_binh", chenh_luong_so_voi_luong_trung_binh_col)

In [41]:
chenh_luong_so_voi_luong_trung_binh_df.show()

+---------+-------------+-----+-----------------------+-----------------------------------+
|phong_ban|ten_nhan_vien|luong|luong_tb_theo_phong_ban|chenh_luong_so_voi_luong_trung_binh|
+---------+-------------+-----+-----------------------+-----------------------------------+
|  Phòng A|  Nhân viên 1| 3000|                 3900.0|                             -900.0|
|  Phòng A|  Nhân viên 2| 4600|                 3900.0|                              700.0|
|  Phòng A|  Nhân viên 3| 4100|                 3900.0|                              200.0|
|  Phòng B|  Nhân viên 4| 2500|     3133.3333333333335|                            -633.33|
|  Phòng B|  Nhân viên 5| 3600|     3133.3333333333335|                             466.67|
|  Phòng B|  Nhân viên 6| 3300|     3133.3333333333335|                             166.67|
|  Phòng C|  Nhân viên 7| 5200|                 5300.0|                             -100.0|
|  Phòng C|  Nhân viên 8| 5400|                 5300.0|                         

### 3. Xác định người có mức lương cao nhất và thấp nhất trong mỗi phòng ban.


In [52]:
luong_cao_nhat_theo_phong_ban = sx_theo_luong_df.where(sx_theo_luong_df["rank_luong_theo_phong_giam_dan"] == 1).drop("rank_luong_theo_phong_giam_dan")

In [53]:
luong_cao_nhat_theo_phong_ban.show()

+---------+-------------+-----+
|phong_ban|ten_nhan_vien|luong|
+---------+-------------+-----+
|  Phòng A|  Nhân viên 2| 4600|
|  Phòng B|  Nhân viên 5| 3600|
|  Phòng C|  Nhân viên 8| 5400|
+---------+-------------+-----+



In [56]:
luong_cao_nhat_theo_phong_ban_2 = sx_theo_luong_df.filter(sx_theo_luong_df["rank_luong_theo_phong_giam_dan"] == 1).drop("rank_luong_theo_phong_giam_dan")

In [57]:
luong_cao_nhat_theo_phong_ban_2.show()

+---------+-------------+-----+
|phong_ban|ten_nhan_vien|luong|
+---------+-------------+-----+
|  Phòng A|  Nhân viên 2| 4600|
|  Phòng B|  Nhân viên 5| 3600|
|  Phòng C|  Nhân viên 8| 5400|
+---------+-------------+-----+



### 4. Tính mức lương lớn thứ hai trong từng phòng ban.


In [58]:
luong_cao_t2_theo_phong_ban = sx_theo_luong_df.where(sx_theo_luong_df["rank_luong_theo_phong_giam_dan"] == 2).drop("rank_luong_theo_phong_giam_dan")

In [59]:
luong_cao_t2_theo_phong_ban.show()

+---------+-------------+-----+
|phong_ban|ten_nhan_vien|luong|
+---------+-------------+-----+
|  Phòng A|  Nhân viên 3| 4100|
|  Phòng B|  Nhân viên 6| 3300|
|  Phòng C|  Nhân viên 7| 5200|
+---------+-------------+-----+



### 5. Sử dụng hàm lead() để so sánh mức lương của mỗi nhân viên với nhân viên tiếp theo trong cùng phòng ban dựa trên thứ tự sắp xếp lương.


In [67]:
luong_nhan_vien_tiep_theo_df = sx_theo_luong_tang_dan_df.withColumn("luong_nhan_su_tiep_theo", lead("luong", 1).over(sx_luong_theo_phong_ban_tang_dan_window))

In [72]:
chenh_luong_so_voi_nhan_vien_tiep_theo_col = luong_nhan_vien_tiep_theo_df["luong_nhan_su_tiep_theo"] - luong_nhan_vien_tiep_theo_df["luong"]

In [73]:
chenh_luong_so_voi_nhan_vien_tiep_theo_df = luong_nhan_vien_tiep_theo_df.withColumn("chenh_luong_so_voi_nhan_vien_tiep_theo", chenh_luong_so_voi_nhan_vien_tiep_theo_col).drop("rank_luong_theo_phong_tang_dan")
chenh_luong_so_voi_nhan_vien_tiep_theo_df.show()

+---------+-------------+-----+-----------------------+--------------------------------------+
|phong_ban|ten_nhan_vien|luong|luong_nhan_su_tiep_theo|chenh_luong_so_voi_nhan_vien_tiep_theo|
+---------+-------------+-----+-----------------------+--------------------------------------+
|  Phòng A|  Nhân viên 1| 3000|                   4100|                                  1100|
|  Phòng A|  Nhân viên 3| 4100|                   4600|                                   500|
|  Phòng A|  Nhân viên 2| 4600|                   NULL|                                  NULL|
|  Phòng B|  Nhân viên 4| 2500|                   3300|                                   800|
|  Phòng B|  Nhân viên 6| 3300|                   3600|                                   300|
|  Phòng B|  Nhân viên 5| 3600|                   NULL|                                  NULL|
|  Phòng C|  Nhân viên 7| 5200|                   5400|                                   200|
|  Phòng C|  Nhân viên 8| 5400|                   

### 6. Sử dụng hàm lag() để so sánh mức lương của mỗi nhân viên với nhân viên trước đó trong cùng phòng ban.

In [76]:
luong_nhan_vien_truoc_do_df = sx_theo_luong_tang_dan_df.withColumn("luong_nhan_vien_truoc_do", lag("luong", 1).over(sx_luong_theo_phong_ban_tang_dan_window))

In [78]:
chenh_luong_so_voi_nhan_vien_truoc_col = luong_nhan_vien_truoc_do_df["luong"] - luong_nhan_vien_truoc_do_df["luong_nhan_vien_truoc_do"]

In [80]:
chenh_luong_so_voi_nhan_vien_truoc_df = luong_nhan_vien_truoc_do_df.withColumn("chenh_luong_so_voi_nhan_vien_truoc", chenh_luong_so_voi_nhan_vien_truoc_col).drop("rank_luong_theo_phong_tang_dan")
chenh_luong_so_voi_nhan_vien_truoc_df.show()

+---------+-------------+-----+------------------------+----------------------------------+
|phong_ban|ten_nhan_vien|luong|luong_nhan_vien_truoc_do|chenh_luong_so_voi_nhan_vien_truoc|
+---------+-------------+-----+------------------------+----------------------------------+
|  Phòng A|  Nhân viên 1| 3000|                    NULL|                              NULL|
|  Phòng A|  Nhân viên 3| 4100|                    3000|                              1100|
|  Phòng A|  Nhân viên 2| 4600|                    4100|                               500|
|  Phòng B|  Nhân viên 4| 2500|                    NULL|                              NULL|
|  Phòng B|  Nhân viên 6| 3300|                    2500|                               800|
|  Phòng B|  Nhân viên 5| 3600|                    3300|                               300|
|  Phòng C|  Nhân viên 7| 5200|                    NULL|                              NULL|
|  Phòng C|  Nhân viên 8| 5400|                    5200|                        

### 7. Xác định phòng ban nào có tổng lương cao nhất.

In [22]:
phong_ban_window = Window.partitionBy("phong_ban")

In [20]:
tong_luong_df = salary_df.withColumn("tong_luong_phong_ban", sum(salary_df["luong"]).over(phong_ban_window))

In [27]:
tong_luong_theo_phong_ban_window = Window.orderBy(desc("tong_luong_phong_ban"))
rank_tong_luong_df = tong_luong_df.withColumn("rank", row_number().over(tong_luong_theo_phong_ban_window))
rank_tong_luong_df.where(rank_tong_luong_df["rank"] == 1).drop("luong", "ten_nhan_vien").show()

+---------+--------------------+----+
|phong_ban|tong_luong_phong_ban|rank|
+---------+--------------------+----+
|  Phòng A|               11700|   1|
+---------+--------------------+----+



In [28]:
# cach khac
tong_luong_theo_phong_ban_df = salary_df.groupBy("phong_ban").agg(sum("luong").alias("tong_luong_phong_ban")).orderBy(desc("tong_luong_phong_ban"))
tong_luong_theo_phong_ban_df.show(1)

+---------+--------------------+
|phong_ban|tong_luong_phong_ban|
+---------+--------------------+
|  Phòng A|               11700|
+---------+--------------------+
only showing top 1 row



### 8. Tính tổng lương lũy kế của từng nhân viên trong phòng ban của họ, sắp xếp theo thứ tự lương tăng dần.

In [34]:
luong_theo_phong_ban_window = Window.partitionBy("phong_ban").orderBy(asc("luong"))
luong_luy_ke_df = salary_df.withColumn("luong_luy_ke", sum(salary_df["luong"]).over(luong_theo_phong_ban))
luong_luy_ke_df.show()

+---------+-------------+-----+------------+
|phong_ban|ten_nhan_vien|luong|luong_luy_ke|
+---------+-------------+-----+------------+
|  Phòng A|  Nhân viên 1| 3000|        3000|
|  Phòng A|  Nhân viên 3| 4100|        7100|
|  Phòng A|  Nhân viên 2| 4600|       11700|
|  Phòng B|  Nhân viên 4| 2500|        2500|
|  Phòng B|  Nhân viên 6| 3300|        5800|
|  Phòng B|  Nhân viên 5| 3600|        9400|
|  Phòng C|  Nhân viên 7| 5200|        5200|
|  Phòng C|  Nhân viên 8| 5400|       10600|
+---------+-------------+-----+------------+



### 9. Sử dụng cume_dist để xác định tỷ lệ phần trăm các nhân viên có mức lương thấp hơn so với mỗi nhân viên trong phòng ban của họ.


In [33]:
''' CUME_DIST()
1. Sắp xếp các giá trị theo thứ tự tăng dần (trong trường hợp của bạn là lương từ thấp đến cao).
2. Xác định vị trí của mỗi giá trị trong dãy giá trị đó.
3. Tính tỷ lệ tổng số giá trị từ đầu cho đến vị trí hiện tại (bao gồm cả vị trí đó), so với tổng số giá trị trong toàn bộ dãy. Tỷ lệ này được biết đến là 
"tỷ lệ tích lũy".
'''
ty_le_tich_luy_df = salary_df.withColumn("ty_le_phan_tram", round(cume_dist().over(luong_theo_phong_ban) * 100, 2))
ty_le_tich_luy_df.show()

+---------+-------------+-----+---------------+
|phong_ban|ten_nhan_vien|luong|ty_le_phan_tram|
+---------+-------------+-----+---------------+
|  Phòng A|  Nhân viên 1| 3000|          33.33|
|  Phòng A|  Nhân viên 3| 4100|          66.67|
|  Phòng A|  Nhân viên 2| 4600|          100.0|
|  Phòng B|  Nhân viên 4| 2500|          33.33|
|  Phòng B|  Nhân viên 6| 3300|          66.67|
|  Phòng B|  Nhân viên 5| 3600|          100.0|
|  Phòng C|  Nhân viên 7| 5200|           50.0|
|  Phòng C|  Nhân viên 8| 5400|          100.0|
+---------+-------------+-----+---------------+



### 10.Sử dụng hàm percent_rank() để xác định xếp hạng phần trăm của mỗi nhân viên trong phòng ban của họ dựa trên mức lương.

In [35]:
xep_hang_phan_tram_df = salary_df.withColumn("xep_hang_phan_tram", percent_rank().over(luong_theo_phong_ban_window))
xep_hang_phan_tram_df.show()

+---------+-------------+-----+------------------+
|phong_ban|ten_nhan_vien|luong|xep_hang_phan_tram|
+---------+-------------+-----+------------------+
|  Phòng A|  Nhân viên 1| 3000|               0.0|
|  Phòng A|  Nhân viên 3| 4100|               0.5|
|  Phòng A|  Nhân viên 2| 4600|               1.0|
|  Phòng B|  Nhân viên 4| 2500|               0.0|
|  Phòng B|  Nhân viên 6| 3300|               0.5|
|  Phòng B|  Nhân viên 5| 3600|               1.0|
|  Phòng C|  Nhân viên 7| 5200|               0.0|
|  Phòng C|  Nhân viên 8| 5400|               1.0|
+---------+-------------+-----+------------------+



In [36]:
ss.stop()